In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import pathlib
import random
import pandas as pd

from credentials import CONNECTION_INFO
from constants import *

import encoders
import db_connect
import helpers

2024-11-29 17:29:17.353198: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-29 17:29:17.362307: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 17:29:17.371870: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 17:29:17.376941: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 17:29:17.388126: I tensorflow/core/platform/cpu_feature_guar

## Config
This notebook has a lot of options to adjust, most of which are controlled here.

In [2]:
ENCODER = encoders.ENCODER_CAESAR
CHUNK_SIZE = 512
PROCESSING_UNITS = CHUNK_SIZE // 4
LAYER_UNITS = max(1, CHUNK_SIZE // 10)

EXTRA_CHECKS = True # Whether to run some (potentially slow) debug checks

INFER_TEXT = False
INFER_KEY = not INFER_TEXT

USE_CUSTOM_LOSS = True
USE_CUSTOM_OUTPUT_ACTIVATION = True

OUTPUT_MIN = 0
OUTPUT_MAX = len(encoders.CHARSET)-1
CUSTOM_LOSS_MODULO = OUTPUT_MAX+1

if INFER_TEXT:
    MAIN_ACCURACY_METRIC = "mae"
    LOSS_METRIC = "mean_squared_error"
    OUTPUT_SIZE = CHUNK_SIZE
    OPTIMIZER = "sgd"
else:
    MAIN_ACCURACY_METRIC = "mae"
    LOSS_METRIC = "mae"
    OPTIMIZER = "adamax"    

    if ENCODER == encoders.ENCODER_CAESAR:
        OUTPUT_SIZE = 1
    elif ENCODER == encoders.ENCODER_SUBST:
        OUTPUT_SIZE = len(encoders.CHARSET)
    else:
        raise Exception(f"Unsupported encoder {ENCODER}")

ENCRYPTED_FILE_LIMIT = -1 # -1 to disable limit

BASE_TRAIN_PCT = 0.75   # Start here. If train or test count would exceed the max, reduce it. Note 0.75 is the default.
MAX_TRAIN_COUNT = 100 # -1 to disable
MAX_TEST_COUNT =  100 # -1 to disable
SPLIT_SEED = 42

LOAD_BEST_MODEL = False # If False, a new model will be created from scratch
SAVE_BEST_MODEL = True
BEST_PATH = './saved_models/best.keras'

# Whether to run the tuner or the hard-coded network build code
TUNE_NETWORK = False
BUILD_NETWORK = not TUNE_NETWORK
TRAIN_MODEL = BUILD_NETWORK

TUNER_DIRECTORY = "tuner_projects"
TUNER_PROJECT_NAME = "KT"

EPOCHS = 15
BATCH_SIZE = 256 # Default is 32 -- going higher speeds things up a LOT, but may cause memory problems

# Data Retrieval and Structuring

In [3]:
# Get database IDs for encoders and key types

encoder_ids= {}
key_type_ids = {}

db = db_connect.DB(CONNECTION_INFO)
with db.get_session() as session:
    for encoder in encoders.ALL_ENCODER_NAMES:
        id = db.get_encoder_id(session, encoder)
        encoder_ids[encoder] = id

    print(f"Encoder IDs: {encoder_ids}")

    for key_type in encoders.KEY_NAMES:
        id = db.get_key_type_id(session, key_type)
        key_type_ids[key_type] = id

    print(f"Key Type IDs: {key_type_ids}")

Encoder IDs: {'None': 1, 'Simplifier': 2, 'Caesar Cipher': 3, 'Substitution Cipher': 4}
Key Type IDs: {'Character Offset': 1, 'Character Map': 2}


In [4]:
# Map source ID to plaintext file (1) details, and source ID to corresponding ciphertext files (1+) details
sid_to_p = {}
sid_to_c = {}

cipher_id = encoder_ids[ENCODER]
with db.get_session() as session:
    # Get all files encrypted with the cipher we care about
    encrypted_files = db.get_files_by_source_and_encoder(session, -1, cipher_id)

    if len(encrypted_files) > ENCRYPTED_FILE_LIMIT and ENCRYPTED_FILE_LIMIT > 0:
        print(f"Found {len(encrypted_files)} encrypted files")
        encrypted_files = random.sample(encrypted_files, ENCRYPTED_FILE_LIMIT)
    print(f"Using {len(encrypted_files)} encrypted files")

    for c in encrypted_files:
        sid = c.source_id
    
        if sid not in sid_to_p:
            plaintext_ids = db.get_files_by_source_and_encoder(session, sid, encoder_ids[encoders.ENCODER_SIMPLIFIER])
            if len(plaintext_ids) != 1:
                raise Exception(f"Found {len(plaintext_ids)} plaintexts for source ID {sid}; should be exactly 1")
            sid_to_p[sid] = plaintext_ids[0]

        if sid not in sid_to_c:
            sid_to_c[sid] = []
        sid_to_c[sid].append(c)

len(sid_to_p), len(sid_to_c)

Using 114 encrypted files


(19, 19)

In [5]:
# Build up the features (X, the cipher texts as offsets) and targets (y, either the plain texts as offsets OR the key).
# Note targets are not necessarily unique.
X = []
y = []

with db.get_session() as session:
    for sid in sid_to_p:
        if INFER_TEXT:
            plaintext = encoders.string_to_offsets(helpers.read_text_file(sid_to_p[sid].path))
            target_chunks = helpers.chunkify(plaintext, CHUNK_SIZE)    
    
        for c in sid_to_c[sid]:
            ciphertext = encoders.string_to_offsets(helpers.read_text_file(c.path))
            feature_chunks = helpers.chunkify(ciphertext, CHUNK_SIZE)

            if INFER_KEY:                
                if ENCODER == encoders.ENCODER_CAESAR:
                    key_value = float(db.get_key_by_id(session, c.key_id).value)

                    if EXTRA_CHECKS:
                        # Decode with the key we got from the DB, make sure it actually works
                        CHECK_CHANCE = 0.1
                        if random.random() < CHECK_CHANCE:
                            plaintext = encoders.string_to_offsets(helpers.read_text_file(sid_to_p[sid].path))
                            plainttext_str = encoders.offsets_to_string(plaintext)
                            ciphertext_str = encoders.offsets_to_string(ciphertext)
                            decoded_str = encoders.decode_caesar(ciphertext_str, int(key_value))
                            if decoded_str != plainttext_str:                                                    
                                print(decoded_str == plainttext_str)
                                print(decoded_str[0:128], plainttext_str[0:128])
                                raise Exception("Decode error")
                    
                elif ENCODER == encoders.ENCODER_SUBST:
                    raise Exception(f"Not yet implemented key for {ENCODER}")
                else:
                    raise Exception(f"Unsupported encoder {ENCODER}")
        
            for i in range (len(feature_chunks)):
                X.append(np.array(feature_chunks[i]).astype(float))

                if INFER_TEXT:
                    y.append(np.array(target_chunks[i]).astype(float))

                if INFER_KEY:
                    y.append(key_value)

X = np.array(X)
y = np.array(y)

X.shape, y.shape

((69396, 512), (69396,))

In [6]:
# Split the preprocessed data into a training and testing dataset
train_count = int(round(len(y) * BASE_TRAIN_PCT))
if train_count > MAX_TRAIN_COUNT and MAX_TRAIN_COUNT > -1:
    print(f"Train count would be {train_count}")
    train_count = int(MAX_TRAIN_COUNT)
print(f"Train count is {train_count}")

test_count = len(y) - train_count
if test_count > MAX_TEST_COUNT and MAX_TEST_COUNT > -1:
    print(f"Test count would be {test_count}")
    test_count = int(MAX_TEST_COUNT)
print(f"Test count is {test_count}")

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_count, test_size=test_count, random_state=SPLIT_SEED)
print( "Initial counts: ", len(X), len(y), len(X_train), len(X_test), len(y_train), len(y_test) )

Train count is 52047
Test count is 17349
Initial counts:  69396 69396 52047 17349 52047 17349


In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((52047, 512), (17349, 512))

# Tensorflow Callbacks
Custom layer activation function, loss and accuracy functions, and model save checkpoint.

In [8]:
# Custom output activation, forcing output to be within range -- but not rounding it off
# Possibly not needed, sigmoid + rescaler might work just as well
def modulo_output(x):
    return tf.math.mod(x, OUTPUT_MAX)

# Custom loss function, adapted from code generated by Copilot
def modulo_distance_loss(y_true, y_pred):
    """ Custom loss function to compute the modulo distance. 
    Args: 
        y_true: True values (ground truth). 
        y_pred: Predicted values. 
        modulo: The modulo value to apply -- hard coded.
    Returns: 
        The computed loss. 
    """ 
    # Compute the raw difference
    diff = tf.abs(y_true - y_pred)
    # Apply modulo operation to handle wrap-around cases
    mod_diff = tf.math.mod(diff, CUSTOM_LOSS_MODULO)
    # Ensure the distance is within the range [0, CUSTOM_LOSS_MODULO/2]
    loss = tf.minimum(mod_diff, CUSTOM_LOSS_MODULO - mod_diff) 
    return tf.reduce_mean(loss)

# Custom accuracy function, counterpart to the loss function above.
# Returns accuracy as 1 - (average percent distance from correct value)
def modulo_distance_accuracy(y_true, y_pred):
    diff = tf.abs(y_true - y_pred)
    mod_diff = tf.math.mod(diff, CUSTOM_LOSS_MODULO)
    loss = tf.minimum(mod_diff, CUSTOM_LOSS_MODULO - mod_diff)

    good_part = tf.math.subtract(CUSTOM_LOSS_MODULO / 2, loss)
    accuracy = tf.math.divide(good_part, CUSTOM_LOSS_MODULO / 2)

    return tf.reduce_mean(accuracy)

# Custom accuracy, percent of correct values after rounding and doing modulo division
def modulo_rounded_accuracy(y_true, y_pred):
    # y_true SHOULD all be round, in-bounds numbers but just in case...
    true_rounded = tf.math.round(y_true)
    true_mod = tf.math.mod(true_rounded, CUSTOM_LOSS_MODULO)

    # y_pred came straight from the model, so it needs to be rounded and mod'ed
    pred_rounded = tf.math.round(y_pred)
    pred_mod = tf.math.mod(pred_rounded, CUSTOM_LOSS_MODULO)

    # Count matches, as a percentage by averaging all the 0's and 1's
    matches_bool = tf.math.equal(true_mod, pred_mod)
    matches_float = tf.cast(matches_bool, tf.float64)
    return tf.reduce_mean(matches_float)

In [9]:
if EXTRA_CHECKS:
    # Testing my loss and accuracy functions
    t_true = [[1.0, 2.0, 3.0, CUSTOM_LOSS_MODULO*5]]*2
    t_pred = [[0.4, 1.5, 3.5, CUSTOM_LOSS_MODULO + 0.4]]*2
    
    t_true_ts = tf.constant(np.array(t_true).astype(float))
    t_pred_ts = tf.constant(np.array(t_pred).astype(float))
    loss = modulo_distance_loss(t_true_ts, t_pred_ts)
    accD = modulo_distance_accuracy(t_true_ts, t_pred_ts)
    accR = modulo_rounded_accuracy(t_true_ts, t_pred_ts)
    print("true:", t_true_ts)
    print("pred:", t_pred_ts)
    print("rond:", tf.math.round(t_pred_ts))
    print("diff", abs(t_true_ts - t_pred_ts))
    print("loss:", loss)
    print("accD:", accD)
    print("accR:", accR)


I0000 00:00:1732930166.095841   13914 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1732930166.177856   13914 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1732930166.177906   13914 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1732930166.180078   13914 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1732930166.180117   13914 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

true: tf.Tensor(
[[ 1.  2.  3. 50.]
 [ 1.  2.  3. 50.]], shape=(2, 4), dtype=float64)
pred: tf.Tensor(
[[ 0.4  1.5  3.5 10.4]
 [ 0.4  1.5  3.5 10.4]], shape=(2, 4), dtype=float64)
rond: tf.Tensor(
[[ 0.  2.  4. 10.]
 [ 0.  2.  4. 10.]], shape=(2, 4), dtype=float64)
diff tf.Tensor(
[[ 0.6  0.5  0.5 39.6]
 [ 0.6  0.5  0.5 39.6]], shape=(2, 4), dtype=float64)
loss: tf.Tensor(0.49999999999999967, shape=(), dtype=float64)
accD: tf.Tensor(0.9000000000000001, shape=(), dtype=float64)
accR: tf.Tensor(0.5, shape=(), dtype=float64)


# Hyperband Tuning

# Model Reload /Creation

In [10]:
if LOAD_BEST_MODEL:
    print(f"Loading model from {BEST_PATH}")
    nn = tf.keras.models.load_model(BEST_PATH)
elif BUILD_NETWORK:
    print("Building new model")
    nn = tf.keras.models.Sequential()

    # Input layer
    nn.add(tf.keras.layers.Embedding(input_dim=CHUNK_SIZE, output_dim=PROCESSING_UNITS, name="Embedding_Input"))

    # This LSTM layer seems to do most of the real work
    nn.add(tf.keras.layers.LSTM(PROCESSING_UNITS, name="LSTM"))

    # Sigmoid layer produces an output between 0 and 1
    # nn.add(tf.keras.layers.Dense(units=PROCESSING_UNITS, activation="sigmoid", name="Sigmoid"))

    # Rescale that 0-1 value from Sigmoid to the correct output range
    # nn.add(tf.keras.layers.Rescaling(scale=OUTPUT_MAX, offset=0, name="Rescaler")) # Input is 0-1

    # Do modulo division to enforce output range limit
    # Note this seems like it should be totally redundant. But the Tuner results suggest that including both
    # mechanisms (Sigmoid + Rescaling, and Modulo Division) produces better results. I don't know why.
    nn.add(tf.keras.layers.Dense(OUTPUT_SIZE, activation=modulo_output, name="Output_Limiter"))

else:
    print("Nothing to do here. Hopefully you got a model somewhere above...")
        
# Check the structure of the model
nn.summary()

Building new model


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Embedding_Input (Embedding)     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ LSTM (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Limiter (Dense)          │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Model Training

In [11]:
%%time

if TRAIN_MODEL:
    # Training checkpoint to save after each epoch, if it is a new best model:
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=BEST_PATH,
        monitor="loss",
        mode="min",
        save_best_only=True,
        save_weights_only=False,
        verbose=1)

    print(f"Training model")

    if USE_CUSTOM_LOSS:
        loss = modulo_distance_loss
        metrics = [modulo_distance_accuracy, modulo_rounded_accuracy]
    else:
        loss = LOSS_METRIC
        metrics = [MAIN_ACCURACY_METRIC]
    
    if SAVE_BEST_MODEL:
        callbacks = [model_checkpoint_callback]
    else:
        callbacks = None
    
    # Compile the Sequential model together and customize metrics
    nn.compile(loss=loss, optimizer=OPTIMIZER, metrics=metrics)
    
    # Fit the model to the training data
    fit_model = nn.fit(X_train_scaled, y_train, epochs=EPOCHS, callbacks=callbacks, batch_size=BATCH_SIZE)

nn.summary()

Training model
Epoch 1/15


2024-11-29 17:29:29.436137: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101


203/204 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 2.0062 - modulo_distance_accuracy: 0.5161 - modulo_rounded_accuracy: 0.1140
Epoch 1: loss improved from inf to 1.75603, saving model to ./saved_models/best.keras
204/204 ━━━━━━━━━━━━━━━━━━━━ 13s 46ms/step - loss: 2.0038 - modulo_distance_accuracy: 0.5161 - modulo_rounded_accuracy: 0.1141
Epoch 2/15
204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 2.5689 - modulo_distance_accuracy: 0.4866 - modulo_rounded_accuracy: 0.1246
Epoch 2: loss did not improve from 1.75603
204/204 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - loss: 2.5686 - modulo_distance_accuracy: 0.4867 - modulo_rounded_accuracy: 0.1246
Epoch 3/15
203/204 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 2.0972 - modulo_distance_accuracy: 0.5135 - modulo_rounded_accuracy: 0.1210
Epoch 3: loss did not improve from 1.75603
204/204 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - loss: 2.0941 - modulo_distance_accuracy: 0.5136 - modulo_rounded_accuracy: 0.1210
Epoch 4/15
204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/s

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Embedding_Input (Embedding)     │ (None, 512, 128)       │        65,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ LSTM (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Limiter (Dense)          │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 591,749 (2.26 MB)

 Trainable params: 197,249 (770.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 394,500 (1.50 MB)

CPU times: user 1min 27s, sys: 17.2 s, total: 1min 44s
Wall time: 2min 20s


In [12]:
# Evaluate the model using the test data

if USE_CUSTOM_LOSS and INFER_KEY:    
    print("Evaluating with model.predict() ...")
    raw_pred = nn.predict(X_test_scaled, batch_size=BATCH_SIZE)

    # My custom loss and accuracy functions are running out of memory for some reason...
    if False:
        y_pred = tf.constant(np.array(raw_pred).astype(np.float64))
        loss = modulo_distance_loss(y_test, y_pred)
        accuracy_distance = modulo_distance_accuracy(y_test, y_pred)
        accuracy_rounded = modulo_rounded_accuracy(y_test, y_pred)
        print(f"Loss: {loss:0.6}, Accuracy (Distance): {accuracy_distance:0.6}, Accuracy (Rounded): {accuracy_rounded:0.6}")

    if INFER_KEY:
        pred_pd = pd.DataFrame(raw_pred)
        print("Inferred key distribution:\n", pred_pd.describe())

print("Evaluating with model.evaluate() ...")
eval_results = nn.evaluate(X_test_scaled, y_test, verbose=2, batch_size=BATCH_SIZE)
print(f"With X_test_scaled, Loss: {eval_results[0]}, Accuracy: {eval_results[1:]}")

if EXTRA_CHECKS:
    # Sometimes the results with totally wacky inputs are so similar to the official "test" data,
    # that it's clearly just plain broken.
    eval_results = nn.evaluate(X_train_scaled, y_train, verbose=2, batch_size=BATCH_SIZE)
    print(f"With X_train_scaled, Loss: {eval_results[0]}, Accuracy: {eval_results[1:]}")
    
    eval_results = nn.evaluate(X_test, y_test, verbose=2, batch_size=BATCH_SIZE)
    print(f"With X_test,        Loss: {eval_results[0]}, Accuracy: {eval_results[1:]}")
    
    eval_results = nn.evaluate(X_test_scaled, y_train[0:len(X_test_scaled)], verbose=2, batch_size=BATCH_SIZE)
    print(f"With mismatch,        Loss: {eval_results[0]}, Accuracy: {eval_results[1:]}")

Evaluating with model.predict() ...
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Result distribution:
                   0
count  17349.000000
mean       4.929049
std        2.705498
min        0.000076
25%        2.951016
50%        4.502173
75%        7.433208
max        8.999927
Evaluating with model.evaluate() ...
68/68 - 1s - 18ms/step - loss: 0.7392 - modulo_distance_accuracy: 0.5062 - modulo_rounded_accuracy: 0.1202
With X_test_scaled, Loss: 0.7391788959503174, Accuracy: [0.5062136650085449, 0.12015396356582642]
204/204 - 3s - 16ms/step - loss: 0.7361 - modulo_distance_accuracy: 0.5067 - modulo_rounded_accuracy: 0.1199
With X_train_scaled, Loss: 0.7360531091690063, Accuracy: [0.5066712498664856, 0.11992713809013367]
68/68 - 1s - 16ms/step - loss: 2.5769 - modulo_distance_accuracy: 0.4631 - modulo_rounded_accuracy: 0.0755
With X_test,        Loss: 2.576937198638916, Accuracy: [0.4630754292011261, 0.07551626861095428]
68/68 - 1s - 16ms/step - loss: 2.4761 - modulo_distance_accuracy: 0.

# Model Usefulness Spot-Check

In [13]:
def decode_chunks_with_model(chunks: list[list], model, scaler, input_already_scaled = True) -> list[list]:
    if input_already_scaled:
        return model.predict(chunks)
    else:
        return model.predict(scaler.transform(chunks))

def decode_text_with_model(ciphertext: str, model, scaler) -> str:
    offset_chunks = helpers.chunkify(encoders.string_to_offsets(ciphertext), CHUNK_SIZE)
    decoded_chunks = decode_chunks_with_model(offset_chunks, model, scaler, input_already_scaled = False)
    rounded = np.rint(decoded_chunks.flatten()).astype(int)
    return encoders.offsets_to_string(rounded)

def infer_key_with_model(ciphertext: str, model, scaler) -> int:
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)
    keys = model.predict(scaler.transform(chunks))
    key = int(round(np.median(keys)))
    return key

if INFER_TEXT:    
    CHUNKS_TO_CHECK = 2
else:
    CHUNKS_TO_CHECK = 20

cipher_file_db = sid_to_c[list(sid_to_c.keys())[0]][0]
ciphertext_path = cipher_file_db.path
ciphertext = helpers.read_text_file(ciphertext_path)
ciphertext = ciphertext[0:CHUNK_SIZE * CHUNKS_TO_CHECK]

if INFER_TEXT:    
    print("Decoded   : ", decode_text_with_model(ciphertext, nn, X_scaler))
if INFER_KEY:
    with db.get_session() as session:
        correct_key = int(db.get_key_by_id(session, cipher_file_db.key_id).value)
    print("Correct Key: ", correct_key)
    
    inferred_key = infer_key_with_model(ciphertext, nn, X_scaler)
    print("Inferred Key: ", inferred_key)
    
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)
    print(nn.predict(scaler.transform(chunks)))
    print(pd.DataFrame(nn.predict(scaler.transform(chunks))).describe())

Correct Key:  9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Inferred Key:  5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[3.6079615e-02]
 [1.2401119e-02]
 [8.9673414e+00]
 [2.1837331e-02]
 [1.5483201e-02]
 [8.9543591e+00]
 [7.5881794e-02]
 [8.9882584e+00]
 [8.9736128e+00]
 [8.9592075e+00]
 [8.9658842e+00]
 [8.9689417e+00]
 [8.9382229e+00]
 [4.8834212e-02]
 [2.2588909e-02]
 [1.3386905e-02]
 [1.7168790e-02]
 [8.9447060e+00]
 [8.9833279e+00]
 [6.9820061e-03]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
               0
count  20.000000
mean    4.495725
std     4.584785
min     0.006982
25%     0.020670
50%     4.507052
75%     8.966249
max     8.988258
